In [2]:
from langchain_community.llms import Ollama
from langchain.agents import Tool
from langchain_community.embeddings import HuggingFaceEmbeddings,OllamaEmbeddings
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader,UnstructuredFileLoader

In [ ]:
loader = DirectoryLoader('docs',loader_cls=UnstructuredFileLoader)

In [9]:
documents = []
l1 = Docx2txtLoader("./potato.docx")
documents.extend(l1.load())
l2=UnstructuredExcelLoader("./data2.xlsx")
documents.extend(l2.load())
documents

[Document(page_content='Cultivation of Potato\n\nAbout Potato:\n\nPotato is the third most important food crop globally.\n\nEconomical crop is known as "Poor Man’s Friend" due to affordability.\n\nMajor source of carbohydrates, often substituting cereals.\n\nRich in essential nutrients: protein, minerals (Ca, P, Fe), vitamins (B1, B2, B6, C), and amino acids (leucine, tryptophane, isoleucine).\n\nCultivated widely in India, especially in Indo-Gangetic plains.\n\nProduction\n\nWest Bengal is a major potato-producing state in India, with Hoogly district leading in both area and production.\n\nClimate:\n\nCool-season crop requiring bright sunshine\n\nLow humidity (60-80%)\n\nTemperatures 15-25°C.\n\nThe optimal night temperature for growth is 21°C.\n\nGrown in winter in plains and summer in hills.\n\nHigh temperatures (>30°C) hinder growth\n\nHumidity and rain can cause diseases\n\nSoil requirements:\n\nFriable, well-aerated, fairly deep, organic matter-rich.\n\nPrefers light soils for un

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
   
)
docs=text_splitter.split_documents(documents)
docs[0]

Document(page_content='Cultivation of Potato\n\nAbout Potato:\n\nPotato is the third most important food crop globally.\n\nEconomical crop is known as "Poor Man’s Friend" due to affordability.\n\nMajor source of carbohydrates, often substituting cereals.\n\nRich in essential nutrients: protein, minerals (Ca, P, Fe), vitamins (B1, B2, B6, C), and amino acids (leucine, tryptophane, isoleucine).\n\nCultivated widely in India, especially in Indo-Gangetic plains.\n\nProduction\n\nWest Bengal is a major potato-producing state in India, with Hoogly district leading in both area and production.\n\nClimate:\n\nCool-season crop requiring bright sunshine\n\nLow humidity (60-80%)\n\nTemperatures 15-25°C.\n\nThe optimal night temperature for growth is 21°C.\n\nGrown in winter in plains and summer in hills.\n\nHigh temperatures (>30°C) hinder growth\n\nHumidity and rain can cause diseases\n\nSoil requirements:\n\nFriable, well-aerated, fairly deep, organic matter-rich.\n\nPrefers light soils for uni

In [11]:
model=Ollama(model="mistral")
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectordb = Chroma.from_documents(documents=docs, 
                                        embedding=embedding,
                                        persist_directory="db")

vectordb.persist()

# Create a retriever from the Chroma vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

template = """You are an friendly ai assistant that help users find the most relevant and accurate answers
to their questions based on the documents you have access to.
When answering the questions, mostly rely on the info in documents.Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
print(chain.invoke("What is the min temperature required in Germination Stage?"))

 Based on the context of the documents you provided, the minimum temperature required during the Germination stage for Potato crops is 10 degrees Celsius.


In [13]:
print(chain.invoke("What are the major disease in Potato?"))

 Based on the provided documents, the major diseases in potato are:

1. Late blight (Control through crop rotation, seed treatment, and fungicides)
2. Black scurf (Control through crop rotation, seed treatment, and fungicides)
3. Bacterial soft rot (Control through crop rotation, seed treatment, and fungicides)
4. Bacterial wilt (Control through crop rotation, seed treatment, and fungicides)
5. Potato scab (Control through crop rotation, seed treatment, and fungicides)

Please note that the documents suggest using crop rotation, seed treatment, and fungicides for controlling these diseases. However, they do not specify which fungicides to use or provide dosage information. It is essential to consult with a potato growing expert or refer to additional, reliable resources for specific recommendations on which fungicides and chemical doses to use for each disease.


In [8]:
print(chain.invoke("What are the processing varities of Potato?"))

 Based on the provided context, there are at least two named potato varieties mentioned for processing purposes: Kufri Frysona and Kufri Chipsona-3. Both of these varieties have good keeping quality and are resistant to late blight, making them suitable for processing into French fries or chips respectively. Their average yields range from 380-400 quintals per hectare.


In [14]:
print(chain.invoke("My name is Aman.Please tell me how to control late blight disease"))

 Based on the information in the documents, here are some ways to control late blight disease in potatoes:

1. Practice crop rotation: Avoid growing potatoes year after year in the same field. Follow rotation with vegetables and green manure.
2. Remove ground creepers and grow late blight resistant varieties like Kufri Jyothi, Kufri Malar, and Kufri Thangam at 45, 60, and 75 days after planting respectively.
3. Spray Mancozeb 2 g/lit or Chlorothalonil 2 g/lit at 45, 60, and 75 days after planting.

These methods can help in the control of late blight disease in potatoes.
